# **ENCODE**

 **Phase 1:** Simple Hunffman Compression

In [ ]:
import heapq
from collections import Counter
from google.colab import drive

# Google Drive'ı mount et
drive.mount('/content/drive')

# Dosya yollarını tanımla
input_file_path = '/content/drive/My Drive/input.txt'
encoded_file_path = '/content/drive/My Drive/encoded.txt'
freq_file_path = '/content/drive/My Drive/frequency.txt'

def calculate_frequencies(file_path):

    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return Counter(text)

def build_huffman_tree(frequencies):

    heap = [[weight, [symbol, ""]] for symbol, weight in frequencies.items()]
    heapq.heapify(heap)
    while len(heap) > 1:
        lo = heapq.heappop(heap)
        hi = heapq.heappop(heap)
        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]
        heapq.heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
    return sorted(heapq.heappop(heap)[1:], key=lambda p: (len(p[1]), p))

def save_freq_table(freq_table, file_path):

    with open(file_path, 'w', encoding='utf-8') as file:
        for symbol, freq in freq_table.items():
            file.write(f'{repr(symbol)}:{freq}\n')

def encode(input_file_path, encoded_file_path, freq_file_path):

    freq_table = calculate_frequencies(input_file_path)
    huffman_tree = build_huffman_tree(freq_table)
    huffman_code = {symbol: code for symbol, code in huffman_tree}
    save_freq_table(freq_table, freq_file_path)

    with open(input_file_path, 'r', encoding='utf-8') as file, open(encoded_file_path, 'w', encoding='utf-8') as output:
        text = file.read()
        encoded_text = ''.join(huffman_code[char] for char in text)
        output.write(encoded_text)

encode(input_file_path, encoded_file_path, freq_file_path)


 **Step 2: **In this step, we convert the bits we obtained through Hunffman compression into decimal values (Formula: bit_sequence = bin(x)[2:].zfill(n)) Here, decimal values corresponding to each 500 bits are used. (N=500, x=/) then we divide the resulting decimal values into groups of 7. The aim here is to connect to the relevant file among the dictionary files kept on the server and print the pixel equivalent of each group of seven digits in the dictionary.

In [ ]:
from google.colab import drive
import os

def decode_bit_sequence(bit_sequence):
    """Verilen bit dizisini decimal bir sayıya çevirir."""
    decimal_value = int(bit_sequence, 2)
    return str(decimal_value)  # Sonucu string olarak döndür

def process_bits_in_chunks(bit_sequence, chunk_size=500, output_dir="/content/drive/My Drive/"):
    """Bit dizisini 500 bitlik parçalara ayırarak işler ve tüm sonuçları yanyana yazdırır."""
    results = []
    lengths = []
    for i in range(0, len(bit_sequence), chunk_size):
        subsequence = bit_sequence[i:i + chunk_size]
        if len(subsequence) < chunk_size:
            print("Remaining bit sequence is shorter than the chunk size.")
            continue
        x = decode_bit_sequence(subsequence)
        results.append(x)
        lengths.append(len(x))

    concatenated_results = ''.join(results)
    formatted_output = [concatenated_results[i:i+7] for i in range(0, len(concatenated_results), 7)]
    formatted_string = '-'.join(formatted_output)

    # Eğer son grup 7 karakterden az ise, o grubu al ve sil
    if len(formatted_output[-1]) < 7:
        key_part = formatted_output.pop(-1)
    else:
        key_part = None

    final_formatted_string = '-'.join(formatted_output)

    # Dosyalara kaydet
    with open(os.path.join(output_dir, 'sevengroup.txt'), 'a') as f:
        f.write(final_formatted_string + '\n')

    with open(os.path.join(output_dir, 'key.txt'), 'a') as f:
        if key_part:
            f.write(key_part + '\n')
        lengths_string = '/'.join(map(str, lengths))
        f.write("Lengths of each part: " + lengths_string + '\n')

    return formatted_string, lengths_string

def process_file(filename, chunk_size=500, output_dir="/content/drive/My Drive/"):
    """Dosyadan bit dizilerini okur ve belirli boyutlardaki parçalara bölerek işler."""
    if os.path.exists(filename):
        with open(filename, 'r') as file:
            for line in file:
                line = line.strip()
                if line:
                    print(f"Processing line: {line}")
                    process_bits_in_chunks(line, chunk_size, output_dir)
    else:
        print("Belirtilen dosya yolu bulunamadı. Lütfen dosya yolunuzu kontrol edin.")

# Google Drive'ı bağla
drive.mount('/content/drive', force_remount=True)

# Dosya yolu örneği
filename = "/content/drive/My Drive/encoded.txt"
process_file(filename)


**Step 3:** We create a meaningless, compressed and information-carrying image data by printing the dictionary-defined hue equivalents of the set of numeric groups of 7.

In [ ]:
from google.colab import drive
from PIL import Image
import os

def load_rgb_values(filename):
    """Dosyadan tüm RGB değerlerini okur ve bir sözlük olarak döndürür."""
    rgb_dict = {}
    with open(filename, 'r') as file:
        for line in file:
            key, value = line.strip().split(':')
            key = key.strip()
            rgb_values = value.strip().strip('()').replace(' ', '').split(',')
            rgb_dict[key] = tuple(map(int, rgb_values))
    return rgb_dict

def process_numbers_and_create_image(numbers, rgb_dict, reverse=False):
    num_pixels = len(numbers)
    width = int(num_pixels ** 0.5)
    height = num_pixels // width if num_pixels % width == 0 else (num_pixels // width) + 1
    image = Image.new('RGB', (width, height))
    pixels = image.load()

    for index, number in enumerate(numbers):
        rgb = rgb_dict.get(number)
        if rgb:
            x, y = index % width, index // width
            pixels[x, y] = rgb

    image_path = '/content/drive/My Drive/output_image2.bmp'
    image.save(image_path, 'BMP')
    print("Image has been saved to:", image_path)
    image.show()

    if reverse:
        print("Reversed RGB values and their corresponding numbers:")
        for number in reversed(numbers):
            rgb = rgb_dict.get(number)
            if rgb:
                print(f"Number {number} has RGB {rgb}.")

def main():
    drive.mount('/content/drive', force_remount=True)
    rgb_file = '/content/drive/My Drive/colab_files/seven_digit_permutations_with_colors.txt'
    rgb_dict = load_rgb_values(rgb_file)

    with open('/content/drive/My Drive/sevengroup.txt', 'r') as file:
        line = file.read().strip()
        numbers = line.split('-')
        reverse = input("Do you want to reverse the numbers and print their RGB values? (yes/no): ")
        process_numbers_and_create_image(numbers, rgb_dict, reverse=reverse.lower() == 'yes')

if __name__ == '__main__':
    main()


At the end of these three steps we will have a BMP file like the one below. https://x.com/IONprotocols/status/1785614955162509605

This image contains Solana's full whitepaper. The original 53-page file has been compressed and transferred into an image. We are trying to integrate a more original version of the codes we have given under the title of image to image in our repository into another image losslessly and we have come a long way in this regard. https://x.com/IONprotocols/status/1784565876361965779 The purpose of this system is to enable users to sell their game assets and experiences to each other through NFTs by enabling the images in NFTs used in game files to carry data. In addition, by adding information codes to the images produced by artificial intelligence, we will solve a major security problem by labeling that these images are not real.


**Where are we in Data Compression (Steno-C ver_2.1)**
What is really exciting about our work is that by changing the form of the information, we can achieve lossless compression at a very high rate. However, since the algorithms we have are still in the testing phase, it is too early to make a clear claim. Therefore, we want to complete all the tests before sharing the algorithms. https://x.com/IONprotocols/status/1785281230637875677


# **DECODE**

All the code in this section reverses the process to get to the actual text

In [ ]:
from google.colab import drive
from PIL import Image
import os

def load_rgb_to_number_map(filename):
    """Dosyadan RGB değerlerini ve karşılık gelen sayıları yükler, bir sözlük döndürür."""
    rgb_map = {}
    with open(filename, 'r') as file:
        for line in file:
            if ':' in line:
                key, value = line.strip().split(':', 1)
                rgb_values = value.split('(')[-1].split(')')[0].strip()
                rgb_map[tuple(map(int, rgb_values.split(',')))] = key.strip()
    return rgb_map

def extract_numbers_from_image(image_path, rgb_map):
    """Resimden RGB değerlerini çıkarır ve karşılık gelen sayıları bulur, aralarında '-' ile ayrılmış bir şekilde döner."""
    image = Image.open(image_path)
    pixels = image.load()
    width, height = image.size
    found_numbers = []

    for y in range(height):
        for x in range(width):
            rgb = pixels[x, y]
            number = rgb_map.get(rgb, None)
            if number:
                found_numbers.append(number)

    # Bulunan sayıları '-' ile birleştirip döndür
    return '-'.join(found_numbers) if found_numbers else ""

def save_numbers_to_file(filepath, numbers):
    """Bulunan sayıları bir dosyaya yazar."""
    with open(filepath, 'w') as file:
        file.write(numbers)

def main():
    # Google Drive'ı bağla
    drive.mount('/content/drive', force_remount=True)

    # RGB haritası yükleniyor
    rgb_map_filename = '/content/drive/My Drive/colab_files/seven_digit_permutations_with_colors.txt'
    rgb_map = load_rgb_to_number_map(rgb_map_filename)

    # Resimden sayıları çıkar
    image_path = '/content/drive/My Drive/output_image2.bmp'
    numbers = extract_numbers_from_image(image_path, rgb_map)

    # Sayıları dosyaya kaydet
    output_filepath = '/content/drive/My Drive/sevengroup.txt'
    save_numbers_to_file(output_filepath, numbers)

if __name__ == '__main__':
    main()


In [ ]:
from google.colab import drive
import os

def reconstruct_bits(decimal_str, lengths):
    """Decimal string değerleri alıp, belirtilen uzunluklara göre bit dizisine dönüştürür."""
    current_pos = 0
    bit_sequence = ''

    for length in lengths:
        part_decimal = decimal_str[current_pos:current_pos + int(length)]
        current_pos += int(length)

        part_bits = bin(int(part_decimal))[2:]  # '0b' ön ekini kaldır
        expected_bits = int(length)
        if len(part_bits) < expected_bits:
            part_bits = part_bits.zfill(expected_bits)

        bit_sequence += part_bits

    return bit_sequence

def process_decimal_file(input_file, output_file, lengths, extra_value):
    if not os.path.exists(input_file):
        print("Girdi dosyası bulunamadı.")
        return

    with open(input_file, 'r') as file:
        decimal_str = file.read().replace('-', '').strip() + str(extra_value)

    bit_sequence = reconstruct_bits(decimal_str, lengths.split('/'))

    with open(output_file, 'w') as file:
        file.write(bit_sequence)

    print("İşlem tamamlandı ve veriler kaydedildi.")

# Google Drive'ı bağla
drive.mount('/content/drive', force_remount=True)

# Dosya yolları
input_filename = "/content/drive/My Drive/sevengroup.txt"
output_filename = "/content/drive/My Drive/output.txt"
key_filename = "/content/drive/My Drive/key.txt"

# key.txt dosyasından değerleri oku
if os.path.exists(key_filename):
    with open(key_filename, 'r') as file:
        contents = file.read()
        pre_length_value = contents.split('Lengths of each part:')[0].strip()
        lengths = contents.split('Lengths of each part:')[1].strip()

    # Fonksiyonu çağır
    process_decimal_file(input_filename, output_filename, lengths, pre_length_value)
else:
    print("Key dosyası bulunamadı.")


In [ ]:
import heapq
import ast
from google.colab import drive

# Google Drive'ı zorla yeniden bağla
drive.mount('/content/drive', force_remount=True)

# Dosya yollarını tanımla
encoded_file_path = '/content/drive/My Drive/output.txt'
freq_file_path = '/content/drive/My Drive/frequency.txt'
decoded_file_path = '/content/drive/My Drive/decoded.txt'

def load_freq_table(file_path):
    """Frekans tablosunu dosyadan yükle."""
    freq_table = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # En son ':' karakterinin pozisyonunu bul
            last_colon_index = line.rfind(':')
            if last_colon_index == -1:
                print(f"Uyarı: '{line.strip()}' satırı ':' karakterini içermiyor ve atlanacak.")
                continue

            # Sembol ve frekansı ayır
            symbol_part = line[:last_colon_index].strip()
            freq_part = line[last_colon_index + 1:].strip()

            try:
                # Frekansı int'e çevir
                freq = int(freq_part)
                # Sembol kısmını doğrudan kullan
                symbol = ast.literal_eval(symbol_part)
                freq_table[symbol] = freq
            except ValueError:
                print(f"Hatalı frekans değeri: '{freq_part}' ({line.strip()}). Bu satır atlanacak.")
            except SyntaxError:
                print(f"Syntax hatası: {line.strip()}")

    return freq_table

def build_huffman_tree(frequencies):
    """Huffman ağacını frekanslara göre oluştur ve kodları döndür."""
    heap = [[weight, [symbol, ""]] for symbol, weight in frequencies.items()]
    heapq.heapify(heap)
    while len(heap) > 1:
        lo = heapq.heappop(heap)
        hi = heapq.heappop(heap)
        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]
        heapq.heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
    return sorted(heapq.heappop(heap)[1:], key=lambda p: (len(p[1]), p))

def decode(encoded_file_path, freq_file_path, decoded_file_path):
    """Kodlanmış metni çöz ve kaydet."""
    freq_table = load_freq_table(freq_file_path)
    huffman_tree = build_huffman_tree(freq_table)
    huffman_code = {code: symbol for symbol, code in huffman_tree}

    with open(encoded_file_path, 'r', encoding='utf-8') as file:
        encoded_text = file.read()

    current_code = ""
    decoded_text = []
    for bit in encoded_text:
        current_code += bit
        if current_code in huffman_code:
            decoded_text.append(huffman_code[current_code])
            current_code = ""

    with open(decoded_file_path, 'w', encoding='utf-8') as file:
        file.write(''.join(decoded_text))

# Dekodlama işlemi için fonksiyonu çağır
decode(encoded_file_path, freq_file_path, decoded_file_path)
